In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyreadstat
import os

In [2]:
pd.set_option('display.max_columns', None)
os.chdir('C:/Users/511232/Desktop/MICS/microdata')
[f for f in os.listdir() if 'sav' in f]

['bh.sav', 'ch.sav', 'fs.sav', 'hh.sav', 'hl.sav', 'wm.sav']

In [10]:
'''data processing prior to generating crosstabs'''

class transform:

    def __init__(self):
        #reading in the .sav files and their metadata files
        os.chdir('C:/Users/511232/Desktop/MICS/microdata')
        df_hh,meta_hh=pyreadstat.read_sav('hh.sav', apply_value_formats=False)
        df_wm,meta_wm=pyreadstat.read_sav('wm.sav', apply_value_formats=False)
        df_hl,meta_hl=pyreadstat.read_sav('hl.sav', apply_value_formats=False)
        
        self.col_names_hh=meta_hh.column_names_to_labels
        self.col_vals_hh=meta_hh.variable_value_labels
        self.col_names_hl=meta_hl.column_names_to_labels
        self.col_vals_hl=meta_hl.variable_value_labels
        self.col_names_wm=meta_wm.column_names_to_labels
        self.col_vals_wm=meta_wm.variable_value_labels

        self.data_hh=df_hh.copy()
        self.data_wm=df_wm.copy()
        self.data_hl=df_hl.copy()

        self.disability_levels={1:'No difficulty',
        2:'Some difficulty',
        3:'A lot of difficulty',
        4:'Cannot do at all'}
        
        self.disability_cols=['AF6','AF8','AF9','AF10','AF11','AF12']
        self.other_cols=['WAGE','HH6','disability','windex5u','windex5r','windex5','MSTATUS','HC14','WB6A','WB5','WB14',
        'MT10','MT11','TA3','CP2','CM17','MN19A','MN19B','MN19H','MN19X','MN19Y','MN19NR','MA6','MT12']

        self.dis_names={'AF6': 'Difficulty seeing, even if wearing glasses or contact lenses',
        'AF8': 'Difficulty hearing, even if using a hearing aid',
        'AF9': 'Difficulty walking or climbing steps',
        'AF10': 'Difficulty remembering or concentrating',
        'AF11': 'Difficulty with self-care, such as washing all over or dressing',
        'AF12': 'Difficulty communicating'}

    def process_data(self):

        os.chdir('C:/Users/511232/Desktop/MICS/Crosstabs')
        ###################### VARIABLE CREATION & MERGES #####################################
        #calculate hh_type variable from HL3 from dataframe df_hl
        def family_type(df):
            nuclear=[1,2,3,13]
            extended=nuclear+[4,5,6,7,8,9,10,11,12]
            composite=extended+[14,96,98]

            if len(df['HL3'])==1:
                df['hh_type']='One person'
            elif all(df['HL3'].isin(nuclear)):
                df['hh_type']='Nuclear'
            elif all(df['HL3'].isin(extended)):
                df['hh_type']='Extended'
            elif all(df['HL3'].isin(composite)):
                df['hh_type']='Composite'
            else:
                df['hh_type']='Unknown'
            return(df)
        
        #create hh_type
        self.data_hl=self.data_hl.groupby(['HH1','HH2']).apply(family_type)
        
        #create age_groups columns
        #agegrp4_1[5-9,10-14,15-19,20-24,25-29,30+]
        cond=[(self.data_hl['HL6']>=5)&(self.data_hl['HL6']<=9),
        (self.data_hl['HL6']>=10)&(self.data_hl['HL6']<=14),
        (self.data_hl['HL6']>=15)&(self.data_hl['HL6']<=19),
        (self.data_hl['HL6']>=20)&(self.data_hl['HL6']<=24),
        (self.data_hl['HL6']>=25)&(self.data_hl['HL6']<=29),
        self.data_hl['HL6']>=30,self.data_hl['HL6'].isna()]
        result=['5-9','10-14','15-19','20-24','25-29','30+','Missing']
        self.data_hl['agegrp4_1']=np.select(cond,result,default='<5')

        #agegrp4_2 [5-9,10-14,15-19,20-24,25-64]
        cond=[(self.data_hl['HL6']>=5)&(self.data_hl['HL6']<=9),
        (self.data_hl['HL6']>=10)&(self.data_hl['HL6']<=14),
        (self.data_hl['HL6']>=15)&(self.data_hl['HL6']<=19),
        (self.data_hl['HL6']>=20)&(self.data_hl['HL6']<=24),
        (self.data_hl['HL6']>=25)&(self.data_hl['HL6']<=64),
        self.data_hl['HL6']>=65,self.data_hl['HL6'].isna()]
        result=['5-9','10-14','15-19','20-24','25-64','65+','Missing']
        self.data_hl['agegrp4_2']=np.select(cond,result,default='<5')

        #age_ict [0-14,15-60,60+]
        cond=[(self.data_hl['HL6']>=0)&(self.data_hl['HL6']<=14),
        (self.data_hl['HL6']>=15)&(self.data_hl['HL6']<=60),
        self.data_hl['HL6']>60,self.data_hl['HL6'].isna()]
        result=['0-14','15-60','60+','Missing']
        self.data_hl['age_ict']=np.select(cond,result,default='<5')

        #agegrp5[<15,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65+]
        cond=[(self.data_hl['HL6']>=15)&(self.data_hl['HL6']<=19),
        (self.data_hl['HL6']>=20)&(self.data_hl['HL6']<=24),
        (self.data_hl['HL6']>=25)&(self.data_hl['HL6']<=29),
        (self.data_hl['HL6']>=30)&(self.data_hl['HL6']<=34),
        (self.data_hl['HL6']>=35)&(self.data_hl['HL6']<=39),
        (self.data_hl['HL6']>=40)&(self.data_hl['HL6']<=44),
        (self.data_hl['HL6']>=45)&(self.data_hl['HL6']<=49),
        (self.data_hl['HL6']>=50)&(self.data_hl['HL6']<=54),
        (self.data_hl['HL6']>=55)&(self.data_hl['HL6']<=59),
        (self.data_hl['HL6']>=60)&(self.data_hl['HL6']<=64),
        self.data_hl['HL6']>=65,self.data_hl['HL6'].isna()]
        result=['15-19','20-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65+','Missing']
        self.data_hl['agegrp5']=np.select(cond,result,default='<15')

        #agegrp15[15-29,30-44,45-64,65+]
        cond=[(self.data_hl['HL6']>=15)&(self.data_hl['HL6']<=29),
        (self.data_hl['HL6']>=30)&(self.data_hl['HL6']<=44),
        (self.data_hl['HL6']>=45)&(self.data_hl['HL6']<=64),
        self.data_hl['HL6']>=65,self.data_hl['HL6'].isna()]
        result=['15-29','30-44','45-64','65+','Missing']
        self.data_hl['agegrp15']=np.select(cond,result,default='<15')

        #agegrp10 [15-24, 25-64, 65+]
        cond=[(self.data_hl['HL6']>=15)&(self.data_hl['HL6']<=24),
        (self.data_hl['HL6']>=25)&(self.data_hl['HL6']<=64),
        self.data_hl['HL6']>=65,self.data_hl['HL6'].isna()]
        result=['15-24','25-64','65+','Missing']
        self.data_hl['agegrp10']=np.select(cond,result,default='<15')

        #agegrp_9 [<18,18-24,25-64,65+,'Not stated']
        cond=[(self.data_hl['HL6']<18),
        (self.data_hl['HL6']>=18)&(self.data_hl['HL6']<=24),
        (self.data_hl['HL6']>=25)&(self.data_hl['HL6']<=64),
        self.data_hl['HL6']>=65,self.data_hl['HL6'].isna()]
        result=['<18','18-24','25-64','65+','Not stated']
        self.data_hl['agegrp_9']=np.select(cond,result,default='<15')

        #agegrp5p age 5+
        cond=[self.data_hl['HL6']>=5,self.data_hl['HL6'].isna()]
        result=['5+','Missing']
        self.data_hl['agegrp5p']=np.select(cond,result,default='<5')

        #agegrp15p age 15+
        cond=[self.data_hl['HL6']>=15,self.data_hl['HL6'].isna()]
        result=['15+','Missing']
        self.data_hl['agegrp15p']=np.select(cond,result,default='<15')

        #agegrp25p age 25+
        cond=[self.data_hl['HL6']>=25,self.data_hl['HL6'].isna()]
        result=['25+','Missing']
        self.data_hl['agegrp25p']=np.select(cond,result,default='<25')

        #create modern method of contraception 'modern_contraceptive'
        '''
        'CP4A': {'?': 'NO RESPONSE', 'A': 'FEMALE STERILIZATION'},
        'CP4B': {'?': 'NO RESPONSE', 'B': 'MALE STERILIZATION'},
        'CP4C': {'?': 'NO RESPONSE', 'C': 'IUD'},
        'CP4D': {'?': 'NO RESPONSE', 'D': 'INJECTABLES'},
        'CP4E': {'?': 'NO RESPONSE', 'E': 'IMPLANTS'},
        'CP4F': {'?': 'NO RESPONSE', 'F': 'PILL'},
        'CP4G': {'?': 'NO RESPONSE', 'G': 'MALE CONDOM'},
        'CP4H': {'?': 'NO RESPONSE', 'H': 'FEMALE CONDOM'},
        'CP4I': {'?': 'NO RESPONSE', 'I': 'DIAPHRAGM'},
        'CP4J': {'?': 'NO RESPONSE', 'J': 'FOAM / JELLY'}'''

        methods=['A','B','C','D','E','F','G','H','I','J']
        cols=['CP4A','CP4B','CP4C','CP4D','CP4E','CP4F','CP4G','CP4H','CP4I','CP4J']
        self.data_wm['modern_contraceptive']=self.data_wm[cols].apply(lambda x: int(any(x.isin(methods))), axis=1)
        self.modern_contraceptive_labels={0:'No', 1:'Yes'}
        self.data_wm['modern_contraceptive']=self.data_wm['modern_contraceptive'].map(self.modern_contraceptive_labels)

        #MERGE [HH ---> WM] on [HH1,HH2] to add 'HC14': 'Household owns the dwelling'
        right_df=self.data_hh[['HH1','HH2','HC14']]
        left_df=self.data_wm
        self.data_wm=pd.merge(left_df,right_df, how='left',on=['HH1','HH2'])
        
        #create hh_size
        self.data_hh['hh_size']=np.where(process.data_hh['HH48']>=8, '8+',process.data_hh['HH48'])
        #create living_alone
        cond=[self.data_hh['HH48']==1,self.data_hh['HH48'].isna(), self.data_hh['HH48']>1]
        result=['alone','Missing','not alone']
        self.data_hh['living_alone']=np.select(cond,result)
        #MERGE [HH ---> HL] to get 'hh_size','living_alone'
        right_df=self.data_hh[['HH1','HH2','hh_size','living_alone']]
        left_df=self.data_hl
        self.df_hl=pd.merge(left_df,right_df, how='left',on=['HH1','HH2'])
        
        #MERGE [HL ---> WM] to add 'hh_size','HL3'(household head relation),'HL6'(age),'hh_type'
        right_df=self.df_hl[['HH1','HH2','HL1','hh_size','HL3','HL6','hh_type',
        'living_alone','agegrp4_1','agegrp4_2','agegrp10','agegrp15','agegrp5p','agegrp15p','agegrp25p','agegrp5',
        'agegrp_9','age_ict']]
        left_df=self.data_wm
        self.data_wm=pd.merge(left_df,right_df, how='left', 
        left_on=['HH1','HH2','LN'], right_on=['HH1','HH2','HL1'])

        #create 'disability_combined' variable. takes the max(code) among ['AF6','AF8','AF9','AF10','AF11','AF12']
        self.data_wm['disability_combined']=self.data_wm[self.disability_cols].apply(lambda x: x.max(), axis=1)
        self.data_wm['disability_combined']=self.data_wm['disability_combined'].map(self.disability_levels)
        #create head of household relationship variable as 1:HH 2:Other 
        self.data_wm['hh_rel']=np.where(self.data_wm['HL3']==1,1,2)
        self.data_wm['hh_rel']=self.data_wm['hh_rel'].map({1:'Head of household', 2:'Other'})

        ###################### LABEL VALUES #########################################
        for col in self.other_cols:
            if col in self.col_vals_hh.keys():
                self.data_wm[col]=self.data_wm[col].map(self.col_vals_hh[col])
                print(f'{col} codes are translated from meta hh')
            elif col in self.col_vals_wm.keys():
                self.data_wm[col]=self.data_wm[col].map(self.col_vals_wm[col])
                print(f'{col} codes are translated from meta women')
            elif col in self.col_vals_hl.keys():
                self.data_wm[col]=self.data_wm[col].map(self.col_vals_hl[col])
                print(f'{col} codes are translated from meta hhl')
            else:
                print(f'!!! WARNING !!! {col} codes were not translated')
                
    ########################################################################################            
    ############################## CROSSTABS################################################
    ########################################################################################

    def generate_xtabs(self):
        os.chdir('C:/Users/511232/Desktop/MICS/Crosstabs')
        
        try:
            '''Table 1:Total population, by sex, age, location and disability status
            'disability_combined' column is calculated by taking the max(code) among ['AF6','AF8','AF9','AF10','AF11','AF12']
            '''
            print('generating Table 1')
            df=self.data_wm.copy()

            xtab=pd.crosstab([df['HH6'],df['disability'],df['disability_combined']],df['WAGE'],
            rownames=['Area','Disability','Disability level'],colnames=['Age'], values=df['wmweight'], aggfunc='sum',dropna=False)      
            xtab.to_excel('Table 1.2 xtab_all_dis_ByAge.xlsx')

            xtab=pd.crosstab([df['disability'],df['disability_combined']],df['HH6'],
            rownames=['Disability','Disability level'],colnames=['Area'], values=df['wmweight'], aggfunc='sum',dropna=False)
            xtab.to_excel('Table 1.1 xtab_all_dis_ByTotalAge.xlsx')
            print('Table 1 generated and saved')
            ###########################################################

            '''Table 2:Persons with disabilities, by type of disability,  sex, age and location
            -generate separate xtabs for all disability_cols
            -stack() them to have a multiindex series and add them to a generator
            -concatenate the generator items
            -stack() and unstack() to get to the final result 
            '''
            print('generating Table 2')
            df_total=[]
            def xtab():
                for col in self.disability_cols:
                    print(f'processing column {col}')
                    r=pd.crosstab([df['HH6'],df['disability'],df[col].map(self.disability_levels)],df['WAGE'],\
                        rownames=['Area','Disability','Level'],colnames=['Age'], values=df['wmweight'], aggfunc='sum').stack()
                    r.name=self.dis_names[col]
                    df_total.append(r)

            #concatenating the series in the resulting generator
            xtab()
            t=pd.concat(df_total, axis=1)
            t['All_disabilities']=t.sum(axis=1)

            #reshape the result
            T=t.stack().unstack([4,3]).sort_index(axis=1, level=0)
            T.to_excel('Table 2 separate disabilites.xlsx')
            print('Table 2 generated and saved')

            ###############################################################
            '''Table 3: Persons with disabilities, by cause of disability, sex and location
            rows: cause of disability ??????????????????????
            columns: disability, HH6 area
            '''
            print('WARNING !!! Table 3 cause of disability not found')
            df=self.data_wm.copy()

            ###############################################################
            '''Table 4: Persons with multi-dimensional disability, by number of functional disability domains, sex and location
            -calculate domain_num by summing the True over the array of disability_cols values
            if the array contains codes (3-a lot of difficulty) or (4-cannot at all) it will result as True
            '''
            print('generating Table 4')
            df=self.data_wm.copy()
            df['domain_num']=df[self.disability_cols].apply(lambda x: sum(x.isin([3,4])), axis=1)
            #generate xtab
            r=pd.crosstab([df['HH6'],df['disability_combined']],df['domain_num'],\
                rownames=['Area','Disability'],colnames=['Number of domains'], values=df['wmweight'], aggfunc='sum', dropna=False)
            
            r.to_excel('Table 4 Number_dis_domain.xlsx')
            print('Table 4 generated and saved')

            #################################################################
            '''Table 5 marital status: Population (15 years and older) marital status, by sex, age, location and disability status'''
            print('generating Table 5')
            df=self.data_wm.copy()
            xtab=pd.crosstab([df['HH6'],df['MSTATUS'],df['MA6'],df['disability'],df['disability_combined']],df['WAGE'],
            rownames=['Area','Marital status','Current marital status','Disability','Disability level'],
            colnames=['Age'], values=df['wmweight'],aggfunc='sum',dropna=False)      

            xtab.to_excel('Table 5 MaritalStatus.xlsx')
            print('Table 5 generated and saved')

            ###############################################################
            '''Table 6 head_HH: Head of household living below the national poverty line and by wealth quintile, 
            sex of head of household, location and disability  status
            1-disability against head of household and othery type of relationship
            -create head of household relationship (in the process_data_wm() )
            df['hh_rel']=np.where(df['HL3']==1,1,2) where 1:HH 2:Other 
            2-disability by head of households by wealth quintiles
            -will generate crosstab among disabled HH with wealth quintiles 
            using windex and not specific windex5u for urban and windex5r for rural since
            they differ from windex5 and will produce contradictory results between urban and rural xtabs
            and the urban and rural disaggregation in the xtab for the total: camp/urban/rural
            '''

            #crosstab 1
            print('generating Table 6.1')
            df=self.data_wm.copy()
            xtab=pd.crosstab([df['HH6'],df['disability'],df['disability_combined']],df['hh_rel'],
            rownames=['Area','Disability','Disability level'],colnames=['HH relationship'], values=df['wmweight'],
            aggfunc='sum',dropna=False)
            xtab.to_excel('Table 6.1 head of HH.xlsx')
            print('Table 6.1 generated and saved')
            
            #crosstab 2
            #filter out the HH
            print('generating Table 6.2')
            df_hh_only=df[df['hh_rel']=='Head of household']
            xtab=pd.crosstab([df_hh_only['HH6'],df_hh_only['disability'],df_hh_only['disability_combined']],df_hh_only['windex5'],
            rownames=['Area','Disability','Disability level'],colnames=['wealth quintile'], values=df_hh_only['wmweight'],
            aggfunc='sum',dropna=False)
            xtab.to_excel('Table 6.2 head of HH_with wquintile.xlsx')
            print('Table 6.2 generated and saved')

            ############################################################################
            '''Table 7 Poorest_type: Poorest persons with disabilities, by type of disability, sex and location
            -filter out the poorest quintile 'windex5' and crosstab with all disability types
            -loop over disability_cols and create crosstabs then stack to end up with multiindex series
            -put them in a generator and concatenate the generator items
            '''
            print('generating Table 7')
            df=self.data_wm.copy()
            #filter out the poorest
            df_poorest=df[df['windex5']=='Poorest'].copy()

            def xtab():
                for col in self.disability_cols:
                    print(f'processing column {col}')
                    r=pd.crosstab([df_poorest['HH6'],df_poorest['disability']],df_poorest[col].map(self.disability_levels),\
                        rownames=['Area','Disability'],colnames=['Disability level'], 
                        values=df_poorest['wmweight'], aggfunc='sum',dropna=False).stack()
                    r.name=self.dis_names[col]
                    yield(r)

            #concatenating the series in the resulting generator
            s=xtab()
            t=pd.concat(s, axis=1)
            t['All_disabilities']=t.sum(axis=1)
            t.to_excel('Table 7 poorest_type.xlsx')
            print('Table 7 generated and saved')

            ###########################################################
            '''Table 8 HH_type&size:
            Households with one or more persons with disabilities (18 years and older), by location and type and size of household
            -data_wm will be filtered according to (age>=18 & disability_combined==3,4) 
            -get the 'HH1','HH2' of the resulting dataframe as a list by zipping both columns
            -filter data resulting from  process_data_wm() on the tuple ('HH1','HH2')

            steps for calculating type of household hh_type (in hl dataframe):
            -grouby hl by ['HH1','HH2']
            -if HL3 isin (1 head,2 spouse/partner,3 son/daughter,13 adopted son daughter)
            if ALL TRUE then code hh_type as 1 Nuclear
            -if HL3 isin (1 head,2 spouse/partner,3 son/daughter,13 adopted son daughter,
            4 son /daughter in law, 5 grnachild, 6 parent, 7 parentin law, 8 brother/sister,
            9, brother/sis in law, 10 uncle/aunt, 11 nephew/niece, 12 other)
            if ALL TRUE then code hh_type as 2 Extended
            if HL3 isin (1 head,2 spouse/partner,3 son/daughter,13 adopted son daughter,
            4 son /daughter in law, 5 grandchild, 6 parent, 7 parent in law, 8 brother/sister,
            9, brother/sis in law, 10 uncle/aunt, 11 nephew/niece, 12 other, 14 servant, 96 other, 98 dont know)
            if ALL TRUE then code hh_type as 3 composite
            WARNING: there is no way to distinguish two nuclear families in a single household from one
            since for example a HH might have 2 spouses or more 
            -data_wm with merge hl left_on=['HH1','HH2','LN'], right_on=['HH1','HH2','HL1'])
            to get the hh_type variable
            -perform corsstab
            '''
            print('generating Table 8')
            df=self.data_wm.copy()
            #criteria 1 for being disabled, and criteria 2 for being >=18
            criteria1=((df['disability_combined']=='Cannot do at all')|(df['disability_combined']=='A lot of difficulty'))
            criteria2=(df['HL6']>=18)

            #filter according to criteria1 & criteria2
            df_filtered=df.loc[criteria1 & criteria2, ['HH1','HH2']].drop_duplicates()
            # filter df_w according to resulting ['HH1','HH2']
            hhd_filter=pd.Series(zip(df['HH1'],df['HH2'])).isin(list(zip(df_filtered['HH1'],df_filtered['HH2'])))
            #filter according to tuple ('HH1','HH2')
            df_wm_filtered=df[hhd_filter]
            
            #generate the crosstab
            xtab=pd.crosstab([df_wm_filtered['HH6'],df_wm_filtered['hh_type']],
            [df_wm_filtered['disability'],df_wm_filtered['disability_combined']],
            rownames=['Area','Household type'],colnames=['Disability','Disability level'], values=df_wm_filtered['wmweight'],
            aggfunc='sum',dropna=False)
            xtab.to_excel('Table 8.1 hh_type_size1.xlsx')
            print('Table 8.1 generated and saved')

            xtab=pd.crosstab([df_wm_filtered['HH6'],df_wm_filtered['hh_type']],df_wm_filtered['hh_size'],
            rownames=['Area','Household type'],colnames=['Household size'], values=df_wm_filtered['wmweight'],
            aggfunc='sum',dropna=False)
            xtab.to_excel('Table 8.2 hh_type_size2.xlsx')
            print('Table 8.2 generated and saved')

            ########################################
            '''Table 9 living_type_age:
            Persons with disabilities living in household or in institution, by type of disability, sex, age and location
            rows: location(HH6), living_alone, hhd_inst (living alone, living with a family in hhd, living in institution)
            note: Palestine doesnt have a question for place of hh whether institution or not so in this case
            the hh_size is being used as alone versus not alone), disability (filter on disabled), disability combined
            columns: separate disabilities (that is on ['AF6','AF8','AF9','AF10','AF11','AF12']), agegroups (WAGE)'''
            
            print('generating Table 9')
            df=self.data_wm.copy()
            #criteria 1 for being disabled, and criteria 2 for being >=18
            criteria=((df['disability_combined']=='Cannot do at all')|(df['disability_combined']=='A lot of difficulty'))

            #filter according to criteria1 & criteria2
            df_filtered=df[criteria].copy()

            #since we have 2 levels of columns, use stack() twice
            def xtab():
                for col in self.disability_cols:
                    print(f'processing column {col}')
                    #generate the crosstab
                    r=pd.crosstab([df_filtered['HH6'],df_filtered['living_alone'],df_filtered['disability'],
                    df_filtered[col]],[df_filtered['agegrp_9']],
                    rownames=['Area','Living alone','Disability','Disability level'],colnames=['Age group'], values=df_filtered['wmweight'],
                    aggfunc='sum',dropna=False).stack()
                    r.name=self.dis_names[col]
                    yield(r)

            #concatenating the series in the resulting generator
            s=xtab()
            t=pd.concat(s, axis=1)

            #reshape the result
            T=t.unstack([4]).sort_index(axis=1, level=0)
            T.to_excel('Table 9 living_type_age.xlsx')
            print('Table 9 generated and saved')

            ####################################################
            '''Table 10 house_ownership: 
            House ownership of population (18 years and older) living alone, by sex, location and disability status
            filter data_wm on 'living_alone' and HL6>=18
            Rows: location (HH6), disability (disability), disability combined
            columns: house ownership (HC14: {1.0: 'OWN', 2.0: 'RENT', 6.0: 'OTHER', 9.0: 'NO RESPONSE'})'''
            
            print('generating Table 10')
            df=self.data_wm.copy()

            criteria=((df['HL6']>=18)&(df['living_alone']=='alone'))
            #filter according to criteria1 & criteria2
            df_filtered=df[criteria].copy()

            #crosstab
            r=pd.crosstab([df_filtered['HH6'],df_filtered['disability'],df_filtered['disability_combined']],
            [df_filtered['HC14']],
            rownames=['Area','Disability','Disability level'],colnames=['house ownership'], values=df_filtered['wmweight'],
            aggfunc='sum',dropna=False)
            r['Total 18+']=r.sum(axis=1)

            r.to_excel('Table 10 house_ownership.xlsx')
            print('Table 10 generated and saved')

            #######################################################
            '''Table 11: Education attainment of population (5 years and older), by sex, age, location and disability status 
            recode HL6 to age_5, to age_15, to age_25
            recode HL6 into age_bounded 15-29, 30-44, 45-64, 65+
            filter data_wm by HL6>=5
            rows: location (HH6), disability (disability), disability combined
            columns: Edu attainment (WB6A), agegroups
            '''
            print('generating Table 11')
            df=self.data_wm.copy()
            df_filtered=df[df['HL6']>=5].copy()

            def xtab():
                for col in ['agegrp5p','agegrp15p','agegrp25p','agegrp15']:
                    #for agegrp5p age 5+
                    r=pd.crosstab([df_filtered['HH6'],df_filtered['disability'],df_filtered['disability_combined']],
                    [df_filtered['WB6A'],df_filtered[col]],
                    rownames=['Area','Disability','Disability level'],colnames=['Edu attainment','Age'], values=df_filtered['wmweight'],
                    aggfunc='sum',dropna=False).stack().stack()
                    r.name=col
                    yield(r)

            #concatenating the series in the resulting generator
            s=xtab()
            t=pd.concat(s, axis=1)

            #reshape the result
            T=t.unstack([4,3]).sort_index(axis=1, level=0)
            T.to_excel('Table 11 living_type_age.xlsx')
            print('Table 11 generated and saved')
        
            #######################################################
            '''Table 12: Education attainment of persons with disabilities (5 years and older), by type of disability, sex, age and location
            filter on age 'HL6']>=5
            rows: ['Area' (HH6),'Disability','Disability level']
            cols: ['Edu attainment' (WB6A),'Age']
            create a new column = disability_cols values
            loop over ['agegrp5p','agegrp15p','agegrp25p'] and yield the result
            loop over disability_cols=['AF6','AF8','AF9','AF10','AF11','AF12'] and yield the result in a generator
            concat all over axis=0
            '''
            print('generating Table 12')
            df=self.data_wm.copy()
            df_filtered=df[df['HL6']>=5].copy()

            df_total=[]
            for dis in self.disability_cols:
                #map the lables to the values
                df_filtered[dis]=df_filtered[dis].map(self.disability_levels)

                df_ages=[]
                for age in ['agegrp5p','agegrp15p','agegrp25p']:

                    r=pd.crosstab([df_filtered['HH6'],df_filtered['disability'],df_filtered[dis]],
                    [df_filtered['WB6A'],df_filtered[age]],
                    rownames=['Area','Disability','Disability level'],colnames=['Edu attainment','Age'], values=df_filtered['wmweight'],
                    aggfunc='sum',dropna=True)
                    #add the disability type as a column
                    r['disability type']=self.dis_names[dis]
                    df_ages.append(r)

                #concatenate on axis=0   
                t=pd.concat(df_ages, axis=0)
                df_total.append(t)

            T=pd.concat(df_total, axis=0)
            #bring the disability type column to the front
            newcols_list=list(T.columns)
            newcols=[newcols_list[-1]]+newcols_list[:-1]
            T=T[newcols]
            T.to_excel('Table 12 EducationAttainment_type.xlsx')

            ################################################################
            '''Table 13: School attendance of population (5 years and older), by sex, age, location and disability status
            filter on age HL6>=5
            rows:['Area' (HH6),school attendance (WB5),'Disability','disability_combined']
            columns:agegrp4_1[5-9,10-14,15-19,20-24,25-29,30+]
            '''
            print('generating Table 13')
            df=self.data_wm.copy()
            df_filtered=df[df['HL6']>=5].copy()

            r=pd.crosstab([df_filtered['HH6'],df_filtered['WB5'],df_filtered['disability'],df_filtered['disability_combined']],
            [df_filtered['agegrp4_1']],
            rownames=['Area','school attendance','Disability','disability_combined'],
            colnames=['Age'], values=df_filtered['wmweight'],
            aggfunc='sum',dropna=False)

            r.to_excel('Table 13 school attendance.xlsx')

            ##############################################################
            '''Table 14 School attendance of persons with disabilities (5 years and older), by type of disability, sex, age and location 
            filter on age HL6>=5
            rows:['Area' (HH6),school attendance (WB5),'Disability','disability_combined']
            columns: disability types, by agegrp4_2 [5-9,10-14,15-19,20-24,25-64]
            loop over disability types ['AF6','AF8','AF9','AF10','AF11','AF12'] to produce xtab with that specific disability type
            then concatenate into a single dataframe
            '''

            print('generating Table 14')
            df=self.data_wm.copy()
            df_filtered=df[df['HL6']>=5].copy()

            df_total=[]
            for dis in self.disability_cols:

                r=pd.crosstab([df_filtered['HH6'],df_filtered['WB5'],df_filtered['disability'],df_filtered['disability_combined']],
                [df_filtered['agegrp4_2']],
                rownames=['Area','school attendance','Disability','Disability level'],colnames=['age'], values=df_filtered['wmweight'],
                aggfunc='sum',dropna=False)
                #add the disability type as a column
                r['disability type']=self.dis_names[dis]
                df_total.append(r)

            #concatenate on axis=0   
            T=pd.concat(df_total, axis=0)
            #append 'disability type' to the row index
            T.set_index('disability type', append=True, inplace=True)
            #make 'disbaility type' a column on top of age
            T=T.unstack(4)
            #swapping the order of column levels between age and disability type
            T.columns=T.columns.swaplevel(0,1)
            #sorting the disability type index
            T=T.sort_index(axis=1, level=0)
            T.to_excel('Table 14 SchoolAttendance_type.xlsx')

            ######################################################################
            '''Table 15: 
            Reasons for not going to/ drop out school for population (5 years and older), by sex, location and disability status
            filter on age HL6>=5 
            rows: ['Area' (HH6),'Disability','disability_combined']
            columns: Reasons for not going to school ???????????????????????????
            '''
            print('WARNING !!! Table 15 Reasons for not going to school not found')

            ######################################################################
            
            '''Table 16: Literacy status for population (15 years and older), by sex, age, location and disability status
            filter on age HL6>=15
            rows: ['Area' (HH6),'Disability','disability_combined']
            columns: WB14 (can read a part of a sentence), agegrp10 [15-24, 25-64, 65+] and another with 15+ to be concatenated
            '''
            print('generating Table 16')
            df=self.data_wm.copy()
            df_filtered=df[df['HL6']>=15].copy()

            df_total=[]
            for age in ['agegrp10','agegrp15p']:
                r=pd.crosstab([df_filtered['HH6'],df_filtered['disability'],df_filtered['disability_combined']],
                [df_filtered['WB14'],df_filtered[age]],
                rownames=['Area','Disability','Disability level'],colnames=['Can read a part of a sentence','age'], 
                values=df_filtered['wmweight'],aggfunc='sum',dropna=False)
                df_total.append(r)
            #concatenate the dataframes
            T=pd.concat(df_total, axis=1)
            T.to_excel('Table 16 Literacy.xlsx')

            ########################################################################

            '''Table 17: Literacy status for persons with disabilities (15 years and older), by type of disability, sex, age and location
            filter on age HL6>=15
            rows: ['Area' (HH6),'Disability type','disability_combined'] loop over disability types and concatenate on axis=0
            columns: WB14 (can read a part of a sentence), agegrp10 [15-24, 25-64, 65+] and another with 15+ to be concatenated 
            '''
            print('generating Table 17')
            df=self.data_wm.copy()
            df_filtered=df[df['HL6']>=15].copy()
            df_total=[]

            for dis in self.disability_cols:
                #map the lables to the values
                df_filtered[dis]=df_filtered[dis].map(self.disability_levels)

                df_ages=[]
                for age in ['agegrp10','agegrp15p']:

                    r=pd.crosstab([df_filtered['HH6'],df_filtered['disability_combined']],
                    [df_filtered['WB14'],df_filtered[age]],
                    rownames=['Area','Disability level'],colnames=['Can read a part of a sentence','Age'],
                    values=df_filtered['wmweight'],aggfunc='sum',dropna=True)
                    #add the disability type as a column
                    r['disability type']=self.dis_names[dis]
                    df_ages.append(r)

                #concatenate on axis=0   
                t=pd.concat(df_ages, axis=0)
                df_total.append(t)

            T=pd.concat(df_total, axis=0)
            #bring the disability type column to the front
            newcols_list=list(T.columns)
            newcols=[newcols_list[-1]]+newcols_list[:-1]
            T=T[newcols]
            T.to_excel('Table 17 Literacy_type_age.xlsx')

            #######################################################################

            '''Table 18: Household education expenditures, by location and disability status
            rows: ['Area' (HH6),'Disability','disability_combined']
            columns: household education expenditure ????????
            '''
            print('generating Table 18')
            df=self.data_wm.copy()
            print('WARNING !!! Table 18 household education expenditure not found')

            #######################################################################

            '''Table 19: Activity status of population (15 years and older), by sex, age, location and disability status
            filter on age HL6>=15
            rows: ['Area' (HH6),'Current activity status' ????????????? ,'Disability','disability_combined']
            columns: agegrp5
            '''

            ########################################################################
            
            '''Table 32: Population with mobile phones and use internet, by sex, age, location and disability status
            rows: ['Area' (HH6),'Disability','disability_combined']
            columns: Own mobile phone ['MT11', age_ict] & ['MT10': 'Internet usage in the last 3 months' , age_ict]
            ['MT12': 'mobile usage in the last 3 months' , age_ict] concatenate both tables
            '''

            print('generating Table 32')
            df=self.data_wm.copy()
            df_total=[]

            multiindex_cols=[]

            for col in ['MT11','MT10','MT12']:
                r=pd.crosstab([df['HH6'],df['disability'],df['disability_combined']],
                [df[col],df['age_ict']],
                rownames=['Area','Disability','Disability level'],colnames=[self.col_names_wm[col],'age'], 
                values=df['wmweight'],aggfunc='sum',dropna=False)
                df_total.append(r)
                #create the multiindex col and append to multiindex_cols
                idx=[]
                for i in r.columns:
                    l=list(i)
                    l.insert(0,self.col_names_wm[col])
                    multiindex_cols.append(tuple(l))
                
            #concatenate the dataframes
            T=pd.concat(df_total, axis=1)
            cols=pd.MultiIndex.from_tuples(multiindex_cols)
            #restructure the columns
            T.columns=cols
                
            #concatenate the dataframes
            T=pd.concat(df_total, axis=1)
            T.to_excel('Table 32 ICT.xlsx')
            #############################################################################

            '''Table 33:Persons with disabilities with mobile phones, by type of disability, sex, age and location
            filter on 'MT11': 'Own a mobile phone' {1.0: 'YES'}
            rows: ['Area' (HH6),'Disability','disability_combined']
            columns: loop over disability_cols=['AF6','AF8','AF9','AF10','AF11','AF12']
            '''

            print('generating Table 33')
            df=self.data_wm.copy()
            df_filtered=df[df['MT11']=='YES'].copy()
            df_total=[]

            for col in self.disability_cols:
                #map the lables to the values
                df_filtered[col]=df_filtered[col].map(self.disability_levels)

                r=pd.crosstab([df_filtered['HH6'],df_filtered['disability']],
                [df_filtered[col]],
                rownames=['Area','Disability'],colnames=['Disability type'], 
                values=df_filtered['wmweight'],aggfunc='sum',dropna=False).stack()
                r.name=self.dis_names[col]
                df_total.append(r)

            #concatenate the dataframes
            T=pd.concat(df_total, axis=1)
            T.to_excel('Table 33 mobilephine_type.xlsx')

            ###############################################################

            '''Table 34:Population receiving social grants/ benefits/ health insurance, by sex, location and disability status
            rows: ['Area' (HH6),'Disability','disability_combined']
            columns: Disability benefits??????????????, Social assistance cash benefits?????????????
            '''

            print('generating Table 34')
            df=self.data_wm.copy()
            print('WARNING !!! Table 34 Disability benefits, Social assistance cash benefits, not found')

            ###############################################################
            
            '''Table 35:Population (15 years and older) who currently use any tobacco product on a 
            daily or non-daily basis, by sex, age, location and disability status
            filter on age HL6>=15 & 'TA3': 'Currently smoking cigarettes'{1.0: 'YES', 2.0: 'NO', 9.0: 'NO RESPONSE'}
            rows: ['Area' (HH6),'Disability','disability_combined']
            columns: 'TA3': 'Currently smoking cigarettes' with agegrp15p, agegrp10
            '''
            print('generating Table 35')
            df=self.data_wm.copy()
            df_filtered=df[(df['HL6']>=15&(df['TA3']==1))].copy()
            df_total=[]
            
            for age in ['agegrp15p','agegrp10']:
                #for the xtab with smoking status
                r1=pd.crosstab([df_filtered['HH6'],df_filtered['disability'],df_filtered['disability_combined']],
                [df_filtered['TA3'],df_filtered[age]],
                rownames=['Area','Disability','Disability level'],colnames=[process.col_names_wm['TA3'],'Disability type'], 
                values=df_filtered['wmweight'],aggfunc='sum',dropna=False)
                df_total.append(r1)

            # concatenate the dataframes
            T=pd.concat(df_total, axis=1)
            T.sort_index(axis=1, level=0)
            T.to_excel('Table 35 Tobacco_use.xlsx')

            #############################################################
            
            '''Table 36:Women of reproductive age (15-49 years) who have their need for family 
            planning satisfied with modern methods, by location and disability status
            filter on age HL6>=15 & HL6<=49
            rows: ['Area' (HH6),'Disability','disability_combined']
            columns: 'CP2': 'Currently using a method to avoid pregnancy'

            modern methods
            'CP4A': {'?': 'NO RESPONSE', 'A': 'FEMALE STERILIZATION'},
            'CP4B': {'?': 'NO RESPONSE', 'B': 'MALE STERILIZATION'},
            'CP4C': {'?': 'NO RESPONSE', 'C': 'IUD'},
            'CP4D': {'?': 'NO RESPONSE', 'D': 'INJECTABLES'},
            'CP4E': {'?': 'NO RESPONSE', 'E': 'IMPLANTS'},
            'CP4F': {'?': 'NO RESPONSE', 'F': 'PILL'},
            'CP4G': {'?': 'NO RESPONSE', 'G': 'MALE CONDOM'},
            'CP4H': {'?': 'NO RESPONSE', 'H': 'FEMALE CONDOM'},
            'CP4I': {'?': 'NO RESPONSE', 'I': 'DIAPHRAGM'},
            'CP4J': {'?': 'NO RESPONSE', 'J': 'FOAM / JELLY'}
            
            traditional methods:
            'CP4K': {'?': 'NO RESPONSE', 'K': 'LACTATIONAL AMENORRHOEA METHOD (LAM)'},
            'CP4L': {'?': 'NO RESPONSE', 'L': 'PERIODIC ABSTINENCE / RHYTHM'},
            'CP4M': {'?': 'NO RESPONSE', 'M': 'WITHDRAWAL'},
            'CP4X': {'?': 'NO RESPONSE', 'X': 'OTHER'}
            '''

            print('generating Table 36')
            df=self.data_wm.copy()
            cond=(df['HL6']>=15) & (df['HL6']<=49)
            df_filtered=df[cond].copy()

            df_total=[]
            r1=pd.crosstab([df_filtered['HH6'],df_filtered['disability'],df_filtered['disability_combined']],
            [df_filtered['CP2']],
            rownames=['Area','Disability','Disability level'],colnames=[self.col_names_wm['CP2']], 
            values=df_filtered['wmweight'],aggfunc='sum',dropna=False)
            #create a multiindex column
            c1=[self.col_names_wm['CP2']]
            c2=list(r1.columns)
            idx=pd.MultiIndex.from_product([c1,c2])
            r1.columns=idx
            df_total.append(r1)

            #filter the currently married and not pregnant or dont know
            cond=((df['MA1']==1) & ((df['CP1']==2) | (df['CP1']==8)))
            df_filtered=df[cond].copy()

            r2=pd.crosstab([df_filtered['HH6'],df_filtered['disability'],df_filtered['disability_combined']],
            [df_filtered['modern_contraceptive']],
            rownames=['Area','Disability','Disability level'],colnames=['Modern contraceptive method'], 
            values=df_filtered['wmweight'],aggfunc='sum',dropna=False)
            c1=['modern_contraceptive']
            c2=list(r2.columns)
            idx=pd.MultiIndex.from_product([c1,c2])
            r2.columns=idx
            df_total.append(r2)

            T=pd.concat(df_total, axis=1)
            T.to_excel('Table 36 family plan.xlsx')

            ###############################################################

            '''Table 37:Births attended by skilled health personnel, by sex, location and disability status
            rows: ['Area' (HH6),'Disability','disability_combined']
            columns: 'CM17': 'Live births in last two years'
            'MN19A': 'Assistance at delivery: Doctor',
            'MN19B': 'Assistance at delivery: Nurse / Midwife',
            'MN19H': 'Assistance at delivery: Relative / Friend',
            'MN19X': 'Assistance at delivery: Other',
            'MN19Y': 'Assistance at delivery: No one',
            'MN19NR': 'Assistance at delivery: No response'
            '''

            print('generating Table 37')
            df=self.data_wm.copy()

            df_total=[]
            for col in ['CM17','MN19A','MN19B','MN19H','MN19X','MN19Y']:

                r=pd.crosstab([df['HH6'],df['disability'],df['disability_combined']],
                [df[col]],
                rownames=['Area','Disability','Disability level'],colnames=[self.col_names_wm[col]],
                values=df['wmweight'],aggfunc='sum',dropna=False)
                #create a multiindex column
                c1=[self.col_names_wm[col]]
                c2=list(r.columns)
                idx=pd.MultiIndex.from_product([c1,c2])
                r.columns=idx
                df_total.append(r)

            T=pd.concat(df_total, axis=1).head()
            T.to_excel('Table 37 skilled health personnel.xlsx')

            ################################################################

            '''Table 38: Women (15-49 years) who make their own informed decisions regarding sexual relations,
            contraceptive use and reproductive health care, by location and disability status
            
            xtab 1
            filter on age HL6>=15 & HL6<=49
            rows: ['Area' (HH6),'Disability','disability_combined']
            columns: 'MA1': 'Currently married or living with a man'

            xtab 2
            filter on age (HL6>=15 & HL6<=49) and 'MA1': {1.0: 'YES, CURRENTLY MARRIED'}
            rows: ['Area' (HH6),'Disability','disability_combined']
            columns: could not find the below
            "for whom decision on health care for themselves is not usually made by the husband/partner or someone else
            " for whom the decision on contraception is not mainly made by the husband/partner
            "who can say no to sex
            '''

            print('generating Table 38')
            df=self.data_wm.copy()
            cond=(df['HL6']>=15) & (df['HL6']<=49)
            df_filtered=df[cond].copy()
            print('WARNING !!! Table 38 related columns, not found')

            ##################################################################

            '''Table 39: Population with large household expenditures on health, by sex, location and disability status
            rows: ['Area' (HH6),'Disability','disability_combined']
            '''

            print('generating Table 39')
            df=self.data_wm.copy()
            cond=(df['HL6']>=15) & (df['HL6']<=49)
            df_filtered=df[cond].copy()
            print('WARNING !!! Table 39 household expenditure or income, not found')


        except Exception as e:
            raise(e)

In [ ]:
process=transform()
process.process_data()

In [34]:
process.generate_xtabs()

generating Table 1
Table 1 generated and saved
generating Table 2
processing column AF6
processing column AF8
processing column AF9
processing column AF10
processing column AF11
processing column AF12
Table 2 generated and saved
WARNING !!! Table 3 cause of disability not found
generating Table 4
Table 4 generated and saved
generating Table 5
Table 5 generated and saved
generating Table 6.1
Table 6.1 generated and saved
generating Table 6.2
Table 6.2 generated and saved
generating Table 7
processing column AF6
processing column AF8
processing column AF9
processing column AF10
processing column AF11
processing column AF12
Table 7 generated and saved
generating Table 8
Table 8.1 generated and saved
Table 8.2 generated and saved
generating Table 9
processing column AF6
processing column AF8
processing column AF9
processing column AF10
processing column AF11
processing column AF12
Table 9 generated and saved
generating Table 10
Table 10 generated and saved
generating Table 11
Table 11 gene

In [12]:
df=process.data_wm.copy()
#criteria 1 for being disabled, and criteria 2 for being >=18
criteria=((df['disability_combined']=='Cannot do at all')|(df['disability_combined']=='A lot of difficulty'))

#filter according to criteria1 & criteria2
df_filtered=df[criteria].copy()

#since we have 2 levels of columns, use stack() twice
def xtab():
    for col in process.disability_cols:
        print(f'processing column {col}')
        #generate the crosstab
        r=pd.crosstab([df_filtered['HH6'],df_filtered['living_alone'],df_filtered['disability'],
        df_filtered[col]],[df_filtered['agegrp_9']],
        rownames=['Area','Living alone','Disability','Disability level'],colnames=['Age group'], values=df_filtered['wmweight'],
        aggfunc='sum',dropna=False).stack()
        r.name=process.dis_names[col]
        yield(r)

#concatenating the series in the resulting generator
s=xtab()
t=pd.concat(s, axis=1)

#reshape the result
T=t.unstack([4]).sort_index(axis=1, level=0)
T

processing column AF6


KeyError: 'agegrp_9'